In [1]:
git_repo = '../../..'

import sys
import os

sys.path.insert(0, os.path.join(git_repo, 'BNP_sensitivity/GMM_clustering/'))

# Linear response libraries
import LinearResponseVariationalBayes as vb
import LinearResponseVariationalBayes.SparseObjectives as obj_lib

# Local libraries
import gmm_clustering_lib as gmm_utils
import modeling_lib 
import functional_sensitivity_lib as fun_sens_lib 
import utils_lib

import matplotlib.pyplot as plt
%matplotlib inline

from copy import deepcopy

import numpy as np
import scipy as sp

np.random.seed(453453)

import json 

In [2]:
# Load results from previous fit.
base_filename = 'ryan_iris_bnp_full_data_fit_alpha8.0.json'
json_input_file = os.path.join(
    git_repo, 'BNP_sensitivity/GMM_clustering/iris_fits/', base_filename)

with open(json_input_file, 'r') as fp:
    fit_dict = json.load(fp)
    model = gmm_utils.get_model_from_checkpoint(fit_dict)
    model_ = gmm_utils.get_model_from_checkpoint(fit_dict)
    best_param = model.global_vb_params.get_free()
    kl_hessian = gmm_utils.get_kl_hessian_from_checkpoint(fit_dict)

ValueError: Wrong size for parameter prior_params.  Expected 15, got 14

In [ ]:
moment_model = gmm_utils.InterestingMoments(model)
dg_deta = moment_model.get_moment_jacobian(best_param)
linear_sens = gmm_utils.LinearSensitivity(model, moment_model, kl_hessian)

In [ ]:
model_refit = deepcopy(model)
print(model.prior_params)

In [ ]:
def refit_with_alpha(new_alpha, model, linear_sens):
    model_refit = deepcopy(model)
    model_refit.prior_params['alpha'].set(new_alpha)
    free_par_refit = linear_sens.predict_from_prior_params(
        model_refit.prior_params.get_free())
    model_refit.optimize_full(free_par_refit,
        init_max_iter=100,
        final_max_iter=500)
    return free_par_refit, model_refit.global_vb_params.get_free()


In [ ]:
alpha_vec = np.linspace(1, 14, num=14)
print(alpha_vec)
free_par_refit_list = []
free_par_refit_lr_list = []
for alpha in alpha_vec:
    print('\n\nalpha: ', alpha)
    free_par_refit, free_par_refit_lr = refit_with_alpha(alpha, model, linear_sens)
    free_par_refit_list.append(free_par_refit)
    free_par_refit_lr_list.append(free_par_refit_lr)
        

In [ ]:
# fit_dict = gmm_utils.get_checkpoint_dictionary(model, kl_hessian, seed=453453)
# base_alpha = model.prior_params['alpha'].get()
# filename = 'ryan_iris_bnp_full_data_fit_alpha{}.json'.format(base_alpha)
# json_output_file = os.path.join(
#     git_repo,
#     'BNP_sensitivity/GMM_clustering/iris_fits/',
#     filename)
# print('saving fit dict to ', json_output_file)
# with open(json_output_file, 'w') as outfile:
#     json.dump(fit_dict, outfile)


In [ ]:
i = 5
print(np.linalg.norm(free_par_refit_list[i]))
plt.plot(free_par_refit_list[i] - best_param, free_par_refit_lr_list[i] - best_param, 'k.')
plt.plot(free_par_refit_list[i] - best_param, free_par_refit_list[i] - best_param, 'r')

In [ ]:
import json
import json_tricks
import checkpoints
import re

np_string = checkpoints.np_string
fit_dict = {}
fit_dict['free_par_refit_list' + np_string] = json_tricks.dumps(free_par_refit_list)
fit_dict['free_par_refit_lr_list' + np_string] = json_tricks.dumps(free_par_refit_lr_list)
fit_dict['alpha_vec' + np_string] = json_tricks.dumps(alpha_vec)
fit_dict['json_input_file'] = json_input_file

output_filename = re.sub('\.json$', '', base_filename) + '_alpha_refit.json'
json_output_file = os.path.join(
    git_repo, 'BNP_sensitivity/GMM_clustering/iris_fits/',
    output_filename)
print(json_output_file)
print('saving fit dict to ', json_output_file)
with open(json_output_file, 'w') as outfile:
    json.dump(fit_dict, outfile)
